In [ ]:
import matplotlib.pyplot as plt
import pyamg
import scipy.sparse.linalg as sla
import numpy as np
import time

from common import set_figure

# Load the sparse matrix

In [ ]:
if 1:
    A = pyamg.gallery.poisson((100,100), format='csr')
    n = A.shape[0]
    b = np.ones((n,))
    x = A * b
    x0 = np.random.rand(n)
if 0:
    #A= sio.mmread('mesh_2d_h005.mtx.gz')
    A = sio.mmread('horseshoe_matrix_anisotropic.mtx.gz')
    n = A.shape[0]
    b = np.ones((n,))
    x = A * b
    x0 = np.random.rand(n)
print(A.nnz)

# Create ILU and LU

Here we will generate the ILU/LU using difference reorderings `permc_spec`

In [ ]:
t = time.time()
B = sla.spilu(A, drop_tol=1e-12, fill_factor=1)#, permc_spec='NATURAL')
tilu = time.time() - t

t = time.time()
C = sla.splu(A.tocsc())#, permc_spec='NATURAL')
tlu = time.time() - t

Mz = lambda r: B.solve(r)
Minv = sla.LinearOperator(A.shape, Mz)

print("nnz in A: %d"%A.nnz)
print("nnz in B: %d"%B.nnz)
print("nnz in C: %d"%C.nnz)
print("time for ILU: %g"%tilu)
print("time for  LU: %g"%tlu)

# Check out the sparsity

In [ ]:
f, ax = plt.subplots(1, 3, figsize=(15,10))
ax[0].spy(A, marker='.', ms=2)
ax[1].spy(B.L + B.U, marker='.', ms=2)
ax[2].spy(C.L + C.U, marker='.', ms=2)

# How fast is the SpMV?

In [ ]:
t = time.time()
Minv.matvec(b)
tspmv = time.time() - t
print("time to solve: %g"%tspmv)

#### What is the equivalant number of SpMVs?

In [ ]:
tilu / tspmv

# RUN GMRES vs PGMRES

In [ ]:
res0 = []
t = time.time()
x = pyamg.krylov.gmres(A, b, x0=x0, tol=1e-8, restrt=20, maxiter=100, M=None, residuals=res0)
t = time.time() - t
print("time for  gmres: %g"%t)
res1 = []
t = time.time()
x = pyamg.krylov.gmres(A, b, x0=x0, tol=1e-8, restrt=20, maxiter=100, M=Minv, residuals=res1)
t = time.time() - t
print("time for pgmres: %g"%t)

In [ ]:
plt.semilogy(res0, lw=3, label='gmres')
plt.semilogy(res1, lw=3, label='pgmres')
plt.legend()

# Now vary the parameters

Two parameters play a role here:
`fill_factor`
and
`drop_tol`

In [ ]:
t = time.time()
B = sla.spilu(A, drop_tol=1e-10, fill_factor=2)
tilu = time.time() - t

Mz = lambda r: B.solve(r)
Minv = sla.LinearOperator(A.shape, Mz)

res2 = []
t = time.time()
x = pyamg.krylov.gmres(A, b, x0=x0, tol=1e-8, restrt=20, maxiter=100, M=Minv, residuals=res2)
t = time.time() - t

print("nnz in A: %d"%A.nnz)
print("nnz in A: %d"%B.nnz)
print("time for    ILU: %g"%tilu)
print("time for pgmres: %g"%t)

plt.semilogy(res0, lw=3, label='gmres')
plt.semilogy(res1, lw=3, label='pgmres')
plt.semilogy(res2, lw=3, label='pgmres again')
plt.legend()

# algebraic multigrid

In [ ]:
t = time.time()
ml = pyamg.smoothed_aggregation_solver(A, max_coarse=10)
resmg = []
x = ml.solve(b, x0=x0, residuals=resmg, accel='gmres', tol=1e-8)
t = time.time() - t

print(t)
res2 = np.array(res2) / res2[0]
resmg = np.array(resmg) / resmg[0]
plt.semilogy(res0, lw=3, label='gmres')
plt.semilogy(res1, lw=3, label='pgmres')
plt.semilogy(res2, lw=3, label='pgmres again')
plt.semilogy(resmg, lw=3, label='AMG')
plt.legend()

In [ ]:
len(resmg)